# pyTorch Convolutional Neural Networks (CNNs)

<div style="padding: 5px; border: 5px solid #a10000ff;">

**Hinweis:** In den Codezellen sind jeweils einige Codeteile nicht programmiert. Diesen Code müssen Sie ergänzen. Die jeweiligen Stellen sind mit einem Kommentar und dem Keyword **TODO** vermerkt und z.T. Stellen mit ... markiert.

Ausserdem gibt es einige assert Statements. Diese geben einen Fehler aus, sollte etwas bei Ihrer Programmierung nicht korrekt sein.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchvision
from torch.utils.data import DataLoader
import os
import random
import PIL.Image as Image


### Torch vorbereiten

In [ ]:
# Wir prüfen ob eine Hardwarebeschleunigung möglich ist und verwenden diese, wenn sie verfügbar ist
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
torch.manual_seed(42) # Setze den Zufallsseed für Reproduzierbarkeit
print(f"Using {device} device")

## Weitere CNNs erstellen

In diesem Notebook werden wir weitere CNNs erstellen. Dabei werden diese CNNs auf eine komplexere Problemstellung angewendet, nämlich auf die Klassifikation von Bildern von Gesichtern in erkannte Emotionen. Das Dataset, welches wir verwenden werden, ist das FER2013 Dataset, welche 48x48 Pixel grosse Bilder enthält und 7 Klassen von Emotionen (0-6) enthält.

Das Dataset wurde von mir in einem passenden Format vorbereitet, damit es direkt in PyTorch verwendet werden kann. 

## Ablauf

<img src="images/cnn2_process.png" alt="CNN Process" width="600"/>

## FER-2013 Dataset vorbereiten, laden und visualisieren

Das FER-2013 Dataset enthält Bilder von Gesichtern, die in verschiedene Emotionen klassifiziert sind. Es ist ein häufig verwendetes Dataset für die Gesichtsemotionserkennung. 
In diesem Abschnitt werden wir das Dataset vorbereiten, laden und einige Beispiele visualisieren.

Ausserdem prüfen wir wie im letzten Notebook die Daten auf mögliche Probleme.

*Datasetquelle: https://www.kaggle.com/datasets/msambare/fer2013*

### Daten laden

In [ ]:
data_path = "datasets/FER-2013/"
data_train_path = os.path.join(data_path, "train")
data_test_path = os.path.join(data_path, "test")

In [ ]:
train_images = torch.load(os.path.join(data_train_path, "train_images.pt"))
train_labels = torch.load(os.path.join(data_train_path, "train_labels.pt"))
test_images = torch.load(os.path.join(data_test_path, "test_images.pt"))
test_labels = torch.load(os.path.join(data_test_path, "test_labels.pt"))


label_mapping = {
    'angry': 0,
    'disgust': 1,
    'fear': 2,
    'happy': 3,
    'sad': 4,
    'surprise': 5,
    'neutral': 6
}

reverse_label_mapping = {v: k for k, v in label_mapping.items()}


### Beispielbilder aus dem FER-2013 Dataset visualisieren

In [ ]:
# 10 Beispiele plotten
fig, axes = plt.subplots(2, 5, figsize=(12, 6))
for i in range(10):
    random_index = random.randint(0, len(train_images) - 1)
    ax = axes[i // 5, i % 5]
    ax.imshow(train_images[random_index].squeeze(), cmap='gray')
    ax.set_title(f"Label: {reverse_label_mapping[train_labels[random_index].item()]}")
    ax.axis('off')
plt.tight_layout()
plt.show()


### Daten validieren

Damit wir die Daten benutzen können, müssen alle Daten gleich formiert sein. Das heisst alle Bilder müssen die gleiche Grösse haben und die Pixelwerte müssen im gleichen Bereich liegen.

#### Pixelwerte überprüfen

In diesem Abschnitt prüfen wir, ob es verschiedene Bildgrössen gibt, ob alle Pixelwerte von 0 bis 255 reichen und zeigen ein Beispiel eines Bildes an.

In [ ]:
# Die Anzahl Trainings- und Testdaten anzeigen
print(f"Anzahl Trainingsbilder: {len(train_images)}")
print(f"Anzahl Testbilder: {len(test_images)}")

# Prüfe ob alle Bilder die gleiche Größe haben. Die Bilder sind als Tensoren gespeichert, daher können wir die Form der Tensoren überprüfen.
image_shapes = [image.shape for image in train_images]
unique_shapes = set(image_shapes)
print(f"Einzigartige Bildgrößen: {unique_shapes}")


#### Prüfen der Pixelwerte

Wir prüfen ob alle Pixelwerte im Bereich von 0 bis 255 liegen. Dies ist wichtig, damit die Bilder korrekt normalisiert werden können und damit das Modell korrekt lernt.

In [ ]:
# Prüfen, dass die Pixelwerte im Bereich von 0 bis 255 liegen. Die Bilder sind als Tensoren gespeichert.
# Wir prüfen die Pixelwerte für alle Bilder, indem wir die Werte der Tensoren extrahieren und in eine flache Liste umwandeln.
temp_train_images_np = train_images.numpy()
assert np.all(temp_train_images_np >= 0) and np.all(temp_train_images_np <= 255)

#### Klassenverteilung und die Labels überprüfen
Wir überprüfen ob alle Klassen sowohl in den Trainings- als auch in den Testdaten vorhanden sind. Es ist wichtig, dass alle Klassen in beiden Datensätzen vertreten sind, damit das Modell lernen kann, alle Klassen zu erkennen und generalisieren kann. Auch prüfen wir, ob keine falschen Labels vorhanden sind, also ob alle Labels tatsächlich zwischen 0 und 6 liegen.

Die Klassenverteilung gibt an, wie viele Bilder es von jeder Klasse (Emotion) im Dataset gibt. Es ist wichtig, die Klassenverteilung zu überprüfen, um sicherzustellen, dass das Dataset ausgewogen ist und keine Klasse überrepräsentiert oder unterrepräsentiert ist. Eine unausgewogene Klassenverteilung kann zu einem Modell führen, das schlecht generalisiert und eine schlechte Leistung auf den Testdaten erzielt.

In [ ]:
# Labels überprüfen
list_train_labels = [int(label.item()) for label in train_labels]
list_test_labels = [int(label.item()) for label in test_labels]
print(f"Einzigartige Labels im Training: {set(list_train_labels)}")
print(f"Einzigartige Labels im Testing: {set(list_test_labels)}")


# Klassenverteilung überprüfen
print("-" * 50)
train_label_counts = pd.Series(list_train_labels).value_counts().sort_index()
test_label_counts = pd.Series(list_test_labels).value_counts().sort_index()

print("Klassenverteilung:")
for label, count in train_label_counts.items():
    print(f"Label {reverse_label_mapping[label]}: {count} Trainingsbilder, {test_label_counts[label]} Testbilder")

### Aufgabe 1: Weitere Validierungsschritte

Auch weitere Validierungsschritte sollten durchgeführt werden.

Prüfen Sie mit Hilfe der Anzeige der Stichproben einige Zellen weiter oben folgende Schritte:

- Sind die Klassen einigermassen gleich verteilt?

> 

- Sind alle Klassen in den Trainings- und Testdaten vorhanden?

> 

- Gibt es fehlende Werte (z.B. auch leere Bilder)? Wie könnten Sie dies feststellen?

> 

- Sind auf allen Bildern tatsächlich Gesichter zu sehen?

> 

- Sind die Ausschnitte der Bilder sinnvoll und ähnlich (z.B. alle Bilder zeigen das Gesicht von vorne, oder gibt es auch Bilder von der Seite)?

> 

- Weshalb prüfen wir die Klassenverteilung?
> 

- Was wäre ein Data Bias der in einem solchen Datensatz auftreten könnte?
> 

- Welche Datenqualitätskriterien haben wir überprüft? Zur Erinnerung nochmals die Liste:
    Vollständigkeit, Konsistenz, - Richtigkeit, - Einzigartigkeit, - Aktualität
> 

### Data Loader erstellen

In diesem Abschnitt erstellen wir einen Data Loader, um die Daten in Batches zu laden und für das Training vorzubereiten. 
Die Daten sind bereits in Trainings- und Testset aufgeteilt.

In [ ]:
# Da die Bilder in einem eigenen Format vorliegen, normalisieren wir die Pixelwerte manuell, indem wir die Pixelwerte durch 255 teilen, um sie in den Bereich von 0 bis 1 zu bringen. Dies ist wichtig, damit das Training stabiler und schneller lernt.

# Wir müssen hier die Bilder normalisieren, da die Pixelwerte im Bereich 0 bis 255 liegen. Wir teilen die Pixelwerte dementsprechend durch 255.
train_images_tensors = train_images.float() / 255.0
test_images_tensors = test_images.float() / 255.0

# Data Loader erstellen
train_loader = DataLoader(list(zip(train_images_tensors, train_labels)), batch_size=64, shuffle=True)
test_loader = DataLoader(list(zip(test_images_tensors, test_labels)), batch_size=64, shuffle=False)


## Neural Network Architektur definieren

In diesem Abschnitt definieren wir die Architektur unseres Convolutional Neural Networks. 

Wir werden mehrere Convolutional Layers, Pooling Layers und Fully Connected Layers verwenden, um ein Modell zu erstellen, das in der Lage ist, die Bilder im FER-2013-Datensatz zu klassifizieren.

### Berechnung der Dimensionen der verschiedenen Layers

In pyTorch müssen wir die Dimensionen der Daten durch die verschiedenen Schichten unseres Netzwerks verfolgen, um die Dimensionen der einzelnen Layers zu konfigurieren.

In diesem Abschnitt berechnen wir die Dimensionen der Daten nach jeder Schicht in unserem CNN.

#### Aufgabe 2

Wir möchten folgende Architektur für unser CNN verwenden:
```
Input (48x48) --> Conv1 (7x7, 12 Kernel, Stride 1, Padding 0) --> Pool1 (2x2, Stride 2) --> Conv2 (5x5, 16 Kernel, Stride 1, Padding 0) --> Pool2 (2x2, Stride 2) --> Flatten --> Fully Connected Layer
```

1. Berechnen Sie die Bildgrösse nach dem ersten Convolutional Layer:

> Hinweis: Überlegen Sie sich wie sich die Bildgrösse mit dem verwendeten Kernel, Padding und Stride verändert.

> 

2. Berechnen Sie die Bildgrösse nach dem ersten Pooling Layer:
> 

3. Berechnen Sie die Bildgrösse nach dem zweiten Convolutional Layer:
> 

4. Berechnen Sie die Bildgrösse nach dem zweiten Pooling Layer:
> 

5. Wie viele Neuronen muss der Fully Connected Layer haben, um die Daten korrekt zu verarbeiten? (Tipp: wir müssen die Anzahl der Kanäle und die Bildgrösse nach dem letzten Pooling Layer berücksichtigen)

> 


In [ ]:
# TODO erstellen Sie mit torch.nn.sequential ein Convolutional Neural Network mit folgenden Schichten:
# - Convolutional Layer mit einem Eingabe-Kanal, 12 Ausgabekanälen, einem Kernel von 7x7, einem Stride von 1 und einem Padding von 0
# - Max Pooling Layer mit einem Kernel von 2x2 und einem Stride von 2
# - LeakyRELU Aktivierungsfunktion
# - Convolutional Layer mit 12 Eingabekanälen, 16 Ausgabekanälen, einem Kernel von 5x5, einem Stride von 1 und einem Padding von 2
# - Max Pooling Layer mit einem Kernel von 2x2 und einem Stride von 2
# - LeakyRELU Aktivierungsfunktion
# - Flatten Layer, um die Daten für den Fully Connected Layer vorzubereiten
# - Fully Connected Layer mit der Anzahl Eingabeneuronen die Sie oben berechnet haben und 10 Ausgabeneuronen.

model_1 = torch.nn.Sequential(
...
)

## Netzwerk trainieren und evaluieren
In diesem Abschnitt trainieren wir unser CNN mit dem MNIST-Datensatz und evaluieren die Leistung des Modells auf dem Testset. Wir werden die Trainings- und Testgenauigkeit berechnen und die Ergebnisse visualisieren.

### Trainingloop

Die Funktion `train_model` trainiert das CNN über eine bestimmte Anzahl von Epochen.

In [ ]:
def train_model(model, train_loader, test_loader, optimizer, loss_fn, max_num_epochs):
    train_losses = []
    test_losses = []

    for epoch in range(max_num_epochs):
        batch_train_losses = []
        batch_test_losses = []
        for batch in train_loader:
            images, labels = batch

            # Wir verschieben die Bilder und Labels auf die gleiche Hardware wie das Modell (CPU oder GPU)
            images, labels = images.to(device), labels.to(device)

            # Wir setzen den Gradienten zurück für den Forward- und Backward-Pass
            optimizer.zero_grad()

            # Wir berechnen die Vorhersagen des Modells für die aktuellen Bilder
            outputs = model(images)

            # Wir berechnen den Loss
            train_loss = loss_fn(outputs, labels)

            #Wir berechnen die Gradienten und aktualisieren die Gewichte des Modells
            train_loss.backward()
            optimizer.step()
            
            batch_train_losses.append(train_loss.item())

        # Testing Loop mit torch.no_grad(), damit wir keine Gradienten berechnen und somit Speicher sparen
        # Wir berechnen den Testloss für die Testdaten aus dem test_loader
        with torch.no_grad():
            for batch in test_loader:
                images, labels = batch
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                batch_test_losses.append(loss_fn(outputs, labels).item())

        training_loss = np.mean(batch_train_losses)
        testing_loss = np.mean(batch_test_losses)
        train_losses.append(training_loss)
        test_losses.append(testing_loss)

        print(f"Epoch {epoch}: Train Loss = {training_loss:.5f}, Test Loss = {testing_loss:.5f}")
    return train_losses, test_losses

In [ ]:
def plot_train_test_losses(train_losses, test_losses):
    # Test und Trainingsverluste visualisieren
    plt.plot(train_losses, label='Trainings Loss')
    plt.plot(test_losses, label='Test Loss')
    plt.xlabel('Epochen')
    plt.ylabel('Loss')
    plt.title('Trainings- und Testverlust über Epochen')
    plt.legend()
    plt.show()

### Modell Trainieren

In [ ]:
# Hyperparameter definieren
max_num_epochs = 20
learning_rate = 0.001

# Optimizer und die Loss-Funktion definieren
optimizer = torch.optim.Adam(model_1.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()

# Das Modell muss noch auf die Hardware verschoben werden.
model_1.to(device)

train_losses, test_losses = train_model(model_1, train_loader, test_loader, optimizer, loss_fn, max_num_epochs)

# Speichern des trainierten Modells
torch.save(model_1.state_dict(), "FER_cnn.pth")

In [ ]:
plot_train_test_losses(train_losses,test_losses)

#### Aufgabe 2.1: Fragen zu den Lernkurven im Plot
Was schliessen Sie aus diesen beiden Lernkurven im Plot?

> 

### Modell laden

In [ ]:
model_1_geladen = torch.nn.Sequential(
    torch.nn.Conv2d(in_channels=1, out_channels=12, kernel_size=7, stride=1, padding=0),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),
    torch.nn.LeakyReLU(),
    torch.nn.Conv2d(in_channels=12, out_channels=16, kernel_size=5, stride=1, padding=0),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),
    torch.nn.LeakyReLU(),
    torch.nn.Flatten(),
    torch.nn.Linear(in_features=16*8*8, out_features=7),
)



# Testen ob das File für das Modell existiert und das Modell geladen werden kann
if os.path.exists("FER_cnn.pth"):
    model_1_geladen.load_state_dict(torch.load("FER_cnn.pth"))
    print("Modell erfolgreich geladen!")


### Funktion um die Modell-Accuracy zu berechnen

In [ ]:
# Accuracy berechnen

def test_model_accuracy(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in test_loader:
            images, labels = batch
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

In [ ]:
print(f"Test Accuracy: {test_model_accuracy(model_1_geladen, test_loader):.2f}%")

## Alternative Modelle erstellen

### Aufgabe 3: Andere Modelle erstellen und evaluieren

In diesem Abschnitt erstellen Sie zwei weitere Modelle mit unterschiedlichen Architekturen und vergleichen Sie die Leistung dieser Modelle mit dem ursprünglichen Modell.

#### Modell 2 erstellen, trainieren und speichern

In [ ]:
# Modell 2: Erstellen Sie ein tieferes CNN mit mehr Filtern und mehr linearen Layers nacheinander. 
# Achten Sie jedoch darauf, dass die Anzahl der Parameter nicht zu gross wird, da ansonsten das Training zu lange dauert.
model_2 = torch.nn.Sequential(
... # TODO Model 2 Architektur hier definieren
)

print("Modell 2 Architektur:")
print(model_2)

In [ ]:
# Modell 2 trainieren
model_2.to(device)
optimizer_2 = torch.optim.Adam(model_2.parameters(), lr=learning_rate)
loss_fn_2 = torch.nn.CrossEntropyLoss()

print("\n=== Training Modell 2 ===")
train_losses_2, test_losses_2 = train_model(model_2, train_loader, test_loader, optimizer_2, loss_fn_2, max_num_epochs)

# Modell 2 speichern
torch.save(model_2.state_dict(), "FER_cnn_model_2.pth")
print("Modell 2 gespeichert!")

plot_train_test_losses(train_losses_2,test_losses_2)

**Oben:** In diesem Diagramm sehen wir wie das Modell extrem auf die Trainingsdaten overfitted und auf den Testdaten wieder schlechter wird.

#### Modell 3 erstellen, trainieren und speichern

In [ ]:
# Modell 3: Ein kompakteres CNN mit weniger Filtern und kleineren Kernel und einem Linear Layer. 
# Eine Andere Aktivierungsfunktion z.B.torch.nn.Sigmoid() oder torch.nn.Tanh() könnte hier verwendet werden. 
model_3 = torch.nn.Sequential(
...# TODO Modell 3 Architektur hier definieren.
)

print("Modell 3 Architektur:")
print(model_3)

In [ ]:
# Modell 3 trainieren
model_3.to(device)
optimizer_3 = torch.optim.Adam(model_3.parameters(), lr=learning_rate)
loss_fn_3 = torch.nn.CrossEntropyLoss()

print("\n=== Training Modell 3 ===")
train_losses_3, test_losses_3 = train_model(model_3, train_loader, test_loader, optimizer_3, loss_fn_3, max_num_epochs)

# Modell 3 speichern
torch.save(model_3.state_dict(), "FER_cnn_model_3.pth")
print("Modell 3 gespeichert!")

# Visualisierung der Trainings- und Testverluste
plot_train_test_losses(train_losses_3,test_losses_3)

**Oben:** In diesem Diagramm sehen wir wie das Modell auf den Trainingsdaten Underfitted. Das Training ist noch nicht abgeschlossen da es zwar einen Abwärtstrend in der Trainings und der Testkurve gibt, aber beide Kurven noch relativ hoch und beieinander liegen.

<div style="padding: 5px; border: 5px solid #a10000ff;">
<h2 style="color: red;">Behalten Sie die Dateien (.pth) der drei gespeicherten Modelle für nächste Woche!</h2>

Wir werden die Modelle nächste Woche, genauer auswerten und benötigen deshalb die Modelle.

### Auswertung der Modelle anzeigen

In [ ]:
# Alle Modelle laden und Accuracy vergleichen
print("\n=== Laden und Evaluieren aller Modelle ===\n")

# ----------------------------------------------------------------------
# Modell 1 laden
# ----------------------------------------------------------------------
model_1_final = torch.nn.Sequential(
    torch.nn.Conv2d(in_channels=1, out_channels=12, kernel_size=7, stride=1, padding=0),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),
    torch.nn.LeakyReLU(),
    torch.nn.Conv2d(in_channels=12, out_channels=16, kernel_size=5, stride=1, padding=0),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),
    torch.nn.LeakyReLU(),
    torch.nn.Flatten(),
    torch.nn.Linear(in_features=16*8*8, out_features=7),
)
if os.path.exists("FER_cnn.pth"):
    model_1_final.load_state_dict(torch.load("FER_cnn.pth"))
    model_1_final.to(device)
    accuracy_1 = test_model_accuracy(model_1_final, test_loader)
    print(f"Modell 1 Test Accuracy: {accuracy_1:.2f}%")

# ----------------------------------------------------------------------
# Modell 2 laden
# ----------------------------------------------------------------------
model_2_final = torch.nn.Sequential(
    ...#TODO Model 2 Architektur hier einfügen
)
if os.path.exists("FER_cnn_model_2.pth"):
    model_2_final.load_state_dict(torch.load("FER_cnn_model_2.pth"))
    model_2_final.to(device)
    accuracy_2 = test_model_accuracy(model_2_final, test_loader)
    print(f"Modell 2 Test Accuracy: {accuracy_2:.2f}%")

# ----------------------------------------------------------------------
# Modell 3 laden
# ----------------------------------------------------------------------

model_3_final = torch.nn.Sequential(
    ...#TODO Model 3 Architektur hier einfügen
)
if os.path.exists("FER_cnn_model_3.pth"):
    model_3_final.load_state_dict(torch.load("FER_cnn_model_3.pth"))
    model_3_final.to(device)
    accuracy_3 = test_model_accuracy(model_3_final, test_loader)
    print(f"Modell 3 Test Accuracy: {accuracy_3:.2f}%")

# Vergleich visualisieren
print("\n=== Zusammenfassung ===")
accuracies = [accuracy_1, accuracy_2, accuracy_3]
models = ['Modell 1', 'Modell 2', 'Modell 3']

plt.figure(figsize=(10, 5))
plt.bar(models, accuracies, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
plt.ylabel('Accuracy (%)', fontsize=12)
plt.title('Vergleich der Test Accuracy aller Modelle', fontsize=14)
plt.ylim([0, 100])
for i, v in enumerate(accuracies):
    plt.text(i, v + 2, f'{v:.2f}%', ha='center', fontsize=11, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.show()

## Optional: Aufgabe 4 mit Grid-Search für Hyperparameter
In diesem Abschnitt können Sie eine Grid-Search durchführen, um die besten Hyperparameter für Ihr Modell zu finden. Sie können verschiedene Werte für die Lernrate, die Anzahl der Epochen, die Batch-Grösse und andere Hyperparameter ausprobieren und die Leistung des Modells vergleichen.

In [ ]:
learning_rates = [0.001, 0.0005, 0.0001]
number_of_epochs = [5, 10, 20]

# Wichtig: Um faire Vergleiche zu ermöglichen, erstellen wir für jede Kombination ein neues Modell (anstatt ein bereits trainiertes Modell weiterzutrainieren)

print("=== Grid Search für Modell 1 Hyperparameter ===\n")
print(f"Teste {len(learning_rates)} Learning Rates: {learning_rates}")
print(f"Teste {len(number_of_epochs)} Epochenzahlen: {number_of_epochs}")
print(f"Gesamtanzahl Kombinationen: {len(learning_rates) * len(number_of_epochs)}\n")

# Dictionary um die Ergebnisse zu speichern
results = []

# Durchlaufe alle Kombinationen von Hyperparametern
for lr in learning_rates:
    for epochs in number_of_epochs:
        print(f"\n{'='*60}")
        print(f"Training mit: Learning Rate = {lr}, Epochs = {epochs}")
        print(f"{'='*60}")
        
        # Erstelle ein neues Modell für jede Kombination (wichtig für faire Vergleiche!)
        model_grid = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1, out_channels=12, kernel_size=7, stride=1, padding=0),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.LeakyReLU(),
            torch.nn.Conv2d(in_channels=12, out_channels=16, kernel_size=5, stride=1, padding=0),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.LeakyReLU(),
            torch.nn.Flatten(),
            torch.nn.Linear(in_features=16*8*8, out_features=7),
        )
        
        # Modell auf Device verschieben
        model_grid.to(device)
        
        # Optimizer und Loss-Funktion mit aktuellen Hyperparametern
        optimizer_grid = torch.optim.Adam(model_grid.parameters(), lr=lr)
        loss_fn_grid = torch.nn.CrossEntropyLoss()
        
        # Modell trainieren
        train_losses_grid, test_losses_grid = train_model(
            model_grid, train_loader, test_loader, optimizer_grid, loss_fn_grid, epochs
        )
        
        # Test Accuracy berechnen
        model_grid.eval()  # Setze Modell in Evaluation Mode
        accuracy = test_model_accuracy(model_grid, test_loader)
        
        # Finaler Loss (der letzten Epoche)
        final_train_loss = train_losses_grid[-1]
        final_test_loss = test_losses_grid[-1]
        
        # Ergebnisse speichern
        results.append({
            'learning_rate': lr,
            'epochs': epochs,
            'accuracy': accuracy,
            'final_train_loss': final_train_loss,
            'final_test_loss': final_test_loss,
            'train_losses': train_losses_grid,
            'test_losses': test_losses_grid
        })
        
        print(f"\nErgebnis: Test Accuracy = {accuracy:.2f}%")
        print(f"Final Train Loss = {final_train_loss:.5f}, Final Test Loss = {final_test_loss:.5f}")

# Finde die beste Kombination basierend auf Test Accuracy
best_result = max(results, key=lambda x: x['accuracy'])

print("\n" + "="*60)
print("=== GRID SEARCH ZUSAMMENFASSUNG ===")
print("="*60)
print("Alle getesteten Kombinationen:")
print(f"{'LR':<12} {'Epochs':<10} {'Accuracy':<12} {'Train Loss':<12} {'Test Loss':<12}")
print("-" * 60)
for r in results:
    print(f"{r['learning_rate']:<12} {r['epochs']:<10} {r['accuracy']:<12.2f} {r['final_train_loss']:<12.5f} {r['final_test_loss']:<12.5f}")

print("\n" + "="*60)
print("BESTE HYPERPARAMETER:")
print(f"  Learning Rate: {best_result['learning_rate']}")
print(f"  Epochs: {best_result['epochs']}")
print(f"  Test Accuracy: {best_result['accuracy']:.2f}%")
print(f"  Final Test Loss: {best_result['final_test_loss']:.5f}")
print("="*60)

# Visualisierung der Ergebnisse
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Accuracy für verschiedene Kombinationen
accuracies = [r['accuracy'] for r in results]
labels = [f"LR={r['learning_rate']}\nE={r['epochs']}" for r in results]
colors = ['green' if r == best_result else 'steelblue' for r in results]

axes[0].bar(range(len(results)), accuracies, color=colors)
axes[0].set_xticks(range(len(results)))
axes[0].set_xticklabels(labels, rotation=45, ha='right')
axes[0].set_ylabel('Test Accuracy (%)')
axes[0].set_title('Grid Search Ergebnisse: Test Accuracy')
axes[0].grid(axis='y', alpha=0.3)
axes[0].axhline(y=best_result['accuracy'], color='red', linestyle='--', 
                label=f"Best: {best_result['accuracy']:.2f}%")
axes[0].legend()

# Plot 2: Train vs Test Loss für beste Konfiguration
axes[1].plot(best_result['train_losses'], label='Train Loss', linewidth=2)
axes[1].plot(best_result['test_losses'], label='Test Loss', linewidth=2)
axes[1].set_xlabel('Epochs')
axes[1].set_ylabel('Loss')
axes[1].set_title(f'Beste Konfiguration: LR={best_result["learning_rate"]}, Epochs={best_result["epochs"]}')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Optional: Bestes Modell nochmals trainieren und speichern
print("\n" + "="*60)
print("(Setzen Sie save_best_model = True um das Modell zu speichern)")
print("="*60)



save_best_model = False  # Ändern Sie dies auf True, um das beste Modell zu speichern

if save_best_model:
    print("\nTrainiere bestes Modell neu...")
    final_best_model = torch.nn.Sequential(
        torch.nn.Conv2d(in_channels=1, out_channels=12, kernel_size=7, stride=1, padding=0),
        torch.nn.MaxPool2d(kernel_size=2, stride=2),
        torch.nn.LeakyReLU(),
        torch.nn.Conv2d(in_channels=12, out_channels=16, kernel_size=5, stride=1, padding=0),
        torch.nn.MaxPool2d(kernel_size=2, stride=2),
        torch.nn.LeakyReLU(),
        torch.nn.Flatten(),
        torch.nn.Linear(in_features=16*8*8, out_features=7),
    )
    final_best_model.to(device)
    
    optimizer_best = torch.optim.Adam(final_best_model.parameters(), lr=best_result['learning_rate'])
    loss_fn_best = torch.nn.CrossEntropyLoss()
    
    train_model(final_best_model, train_loader, test_loader, optimizer_best, loss_fn_best, best_result['epochs'])
    
    torch.save(final_best_model.state_dict(), "FER_cnn_best_gridsearch.pth")
    print(f"Bestes Modell gespeichert als: FER_cnn_best_gridsearch.pth")
    print(f"Verwendete Hyperparameter: LR={best_result['learning_rate']}, Epochs={best_result['epochs']}")
